In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
!pwd

/content


* 텐서플로 세션(Session)은 일종의 실행창
  * 참조 : https://chan-lab.tistory.com/6

* tensorflow 로 학습시킨 딥러닝 모델을 저장하는 방법중 하나로 Checkpoint 을 이용하는 방법
  * 참조 : http://jaynewho.com/post/8

In [4]:
import numpy as np
import os
import glob
from imageio import imread, imsave
import cv2

MODEL_PATH = os.path.join('drive','MyDrive','Colab Notebooks','datasets','makeup','models','model_p2','model.meta')
MODEL_CHECKPOINT_PATH = os.path.join('drive','MyDrive','Colab Notebooks','datasets','makeup','models','model_p2')

def preprocess(img):
  return (img/255. - 0.5) *2 

def deprocess(img):
  return (img+1)/2

def model_predict():
  batch_size = 1
  img_size=256
  no_makeup = cv2.resize(imread(os.path.join('drive','MyDrive','Colab Notebooks','datasets','makeup','mlib','imgs','no_makeup','xfsy_0068.png')),(img_size, img_size))
  print('no_makeup.shape:',no_makeup.shape)
  X_img = np.expand_dims(preprocess(no_makeup), 0)
  print('X_img.shape:',X_img.shape)
  makeups = glob.glob(os.path.join('drive','MyDrive','Colab Notebooks','datasets','makeup','mlib','imgs','makeup','*.*'))
  result = np.ones((2*img_size, (len(makeups)+1)*img_size,3))
  result[img_size: 2*img_size, :img_size] = no_makeup / 255.

  # 현재 그래프에 이어 붙는 형식을 피하려면 tf.reset_default_graph()를 실행해서 default graph로 초기화
  tf.reset_default_graph()

  # Session 선언문
  sess = tf.Session()
  # 변수 초기화
  sess.run(tf.global_variables_initializer())

  # 전체 모델을 내보내고 가져오는 API/ .meta파일을 불러오기 위해서는 tf.train.import_meta_graph() 함수를 이용
  saver = tf.train.import_meta_graph(MODEL_PATH)
  saver.restore(sess, tf.train.latest_checkpoint(MODEL_CHECKPOINT_PATH))

  graph = tf.get_default_graph()
  X = graph.get_tensor_by_name('X:0')
  Y = graph.get_tensor_by_name('Y:0')
  Xs = graph.get_tensor_by_name('generator/xs:0')

  for i in range(len(makeups)):
    makeup = cv2.resize(imread(makeups[i]), (img_size,img_size))
    Y_img = np.expand_dims(preprocess(makeup),0)
    Xs_ = sess.run(Xs, feed_dict={X: X_img, Y: y_img})
    Xs_ = deprocess(Xs_)
    result[:img_size, (i+1)*img_size:(i+2)*img_size] = makeup /255.
    result[img_size:2*img_size, (i+1)*img_size:(i+2)*img_size]=Xs_[0]

    imsave('results.jpg', result)
  return result

In [5]:
model_predict()

no_makeup.shape: (256, 256, 3)
X_img.shape: (1, 256, 256, 3)
INFO:tensorflow:Restoring parameters from drive/MyDrive/Colab Notebooks/datasets/makeup/models/model_p2/model


OutOfRangeError: ignored